**필요한 Library Import 및 Install**

In [1]:
import numpy as np

import tensorflow as tf

!pip install tensorflow-hub
!pip install tfds-nightly
import tensorflow_hub as hub
import tensorflow_datasets as tfds

     |████████████████████████████████| 4.0MB 7.2MB/s 


**IMDB 데이터셋 로드하기**

tfds -> tensorflow_datasets 라이브러리를 의미하며 해당 라이브러리를 통해 TF에서 바로 사용 가능한 Dataset Collection을 이용할 수 있음

In [3]:
train_data, validation_data, test_data = tfds.load(name="imdb_reviews", split=('train[:60%]', 'train[60%:]', 'test'), as_supervised=True)

Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


**Dataset은 Preprocessing된 정수 배열로 이루어져 있음**

각 정수는 단어를 나타내며, Label은 정수 0 (부정적 리뷰), 1 (긍정적 리뷰) 2가지로 이루어짐

In [4]:
train_examples_batch, train_labels_batch = next(iter(train_data.batch(10)))
train_examples_batch

<tf.Tensor: shape=(10,), dtype=string, numpy=
array([b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it.",
       b'I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However on this occasion I fell 

In [5]:
train_labels_batch

<tf.Tensor: shape=(10,), dtype=int64, numpy=array([0, 0, 0, 1, 1, 1, 0, 0, 0, 0])>

**모델 구성하기**

신경망 (Neural Network)는 Layer 구조로 이루어져 있음

  * 중요한 구조적 결정 3가지

    1. 어떻게 텍스트를 표현할 것인가?
    2. 모델에서 얼마나 많은 층 (Layer)를 사용할 것인가?
    3. 각 층에서 얼마나 많은 Hidden Unit (은닉 유닛)을 사용할 것인가?

  * 해당 Model은 문장 데이터로 구성되어 있기 때문에 텍스트를 어떻게 표현할 지 고민해야 함

  * 텍스트 표현 방식 중 하나인 Embedding Vector 변환

> 첫 번째 Layer에 Pre-trained된 Text Embedding 사용 가능 
 * 해당 예제에서는 TF Hub의 Pre-trained된 Text Embedding 모델인 gnews-swivel-20dim을 사용하고 있음 (이외에도 다양한 Text Embedding Model 존재)


   

In [6]:
# TF Hub의 Embedding Model을 사용하는 Keras Layer
# 해당 모델은 하나의 문장을 구분이 가능한 Token으로 나누고, 각 Token의 Embedding을 연결하여 Return함
embedding = "https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1"
hub_layer = hub.KerasLayer(embedding, input_shape=[], dtype=tf.string, trainable=True)
hub_layer(train_examples_batch[:3])

<tf.Tensor: shape=(3, 20), dtype=float32, numpy=
array([[ 1.765786  , -3.882232  ,  3.9134233 , -1.5557289 , -3.3362343 ,
        -1.7357955 , -1.9954445 ,  1.2989551 ,  5.081598  , -1.1041286 ,
        -2.0503852 , -0.72675157, -0.65675956,  0.24436149, -3.7208383 ,
         2.0954835 ,  2.2969332 , -2.0689783 , -2.9489717 , -1.1315987 ],
       [ 1.8804485 , -2.5852382 ,  3.4066997 ,  1.0982676 , -4.056685  ,
        -4.891284  , -2.785554  ,  1.3874227 ,  3.8476458 , -0.9256538 ,
        -1.896706  ,  1.2113281 ,  0.11474707,  0.76209456, -4.8791065 ,
         2.906149  ,  4.7087674 , -2.3652055 , -3.5015898 , -1.6390051 ],
       [ 0.71152234, -0.6353217 ,  1.7385626 , -1.1168286 , -0.5451594 ,
        -1.1808156 ,  0.09504455,  1.4653089 ,  0.66059524,  0.79308075,
        -2.2268345 ,  0.07446612, -1.4075904 , -0.70645386, -1.907037  ,
         1.4419787 ,  1.9551861 , -0.42660055, -2.8022065 ,  0.43727064]],
      dtype=float32)>

In [8]:
# 앞서 생성한 Layer를 포함한 전체 Model을 생성
model = tf.keras.Sequential()

# 앞에서 생성한 Layer 추가
model.add(hub_layer)

# tf.keras.layers.Dense 명령어는 Input을 넣으면 Output으로 바꿔주는 역할을 한다고 보면 됨 
# 즉 모델은 이러한 Layer를 하나 씩 거쳐가면서 Output을 만든다? -> 소프트웨어공학의 Layered Architecture와 유사하다고 보면 될 듯
# 활성화 함수는 https://yeomko.tistory.com/39 참고

# 16 = units (출력 값 크기), activation='relu' = 활성화 함수로 'ReLU' 사용
# 앞선 Layer에서 나온 고정된 크기의 Output Vector들은 16개의 Hidden nit을 가진 Dense Layer로 주입
model.add(tf.keras.layers.Dense(16, activation='relu'))

# 하나의 출력 Node를 가진 Dense로서, sigmoid 활성화 함수를 사용해 확률 또는 신뢰도 수준을 표현하는 0~1 사이의 실수 출력됨
model.add(tf.keras.layers.Dense(1))

# 생성된 모델 Summary 출력
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     (None, 20)                400020    
_________________________________________________________________
dense_2 (Dense)              (None, 16)                336       
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 17        
Total params: 400,373
Trainable params: 400,373
Non-trainable params: 0
_________________________________________________________________


**Loss Function & Optimizer**

Model의 Train을 위해서는 Loss Function과 Optimizer가 필요함

우리가 다루는 문제가 이진 분류 문제이고 모델이 확률을 출력하고 있음
 
  -> Loss Function으로 binary_crossentropy 사용 (https://hororolol.tistory.com/375 참조)

  -> Optimizer..는 최적화를 위한 것 같긴 한데 잘 모르겠다 (https://sacko.tistory.com/42 참조)



In [9]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

**Model Train**

512개의 Sample로 이루어진 Mini-Batch에서 20번의 Epoch 동안 훈련

  -> x_train과 y_train에 Tensor에 있는 모든 Sample에 대해 20번 반복한다는 의미

In [10]:
history = model.fit(train_data.shuffle(10000).batch(512),
                    epochs=20,
                    validation_data=validation_data.batch(512),
                    verbose=1)

Epoch 1/20
30/30 [==============================] - 4s 89ms/step - loss: 0.9927 - accuracy: 0.4803 - val_loss: 0.8028 - val_accuracy: 0.4559
Epoch 2/20
30/30 [==============================] - 2s 70ms/step - loss: 0.7653 - accuracy: 0.4779 - val_loss: 0.7245 - val_accuracy: 0.4999
Epoch 3/20
30/30 [==============================] - 2s 70ms/step - loss: 0.6981 - accuracy: 0.5151 - val_loss: 0.6711 - val_accuracy: 0.5364
Epoch 4/20
30/30 [==============================] - 2s 72ms/step - loss: 0.6482 - accuracy: 0.5727 - val_loss: 0.6257 - val_accuracy: 0.6103
Epoch 5/20
30/30 [==============================] - 2s 72ms/step - loss: 0.6002 - accuracy: 0.6478 - val_loss: 0.5824 - val_accuracy: 0.6582
Epoch 6/20
30/30 [==============================] - 2s 71ms/step - loss: 0.5537 - accuracy: 0.6930 - val_loss: 0.5417 - val_accuracy: 0.7095
Epoch 7/20
30/30 [==============================] - 2s 71ms/step - loss: 0.5095 - accuracy: 0.7363 - val_loss: 0.5045 - val_accuracy: 0.7281
Epoch 8/20
30

**Model Evaluation**

모델 성능 확인 - 손실 및 정확도 반환

이보다 고급 방법을 사용하면 95%에 가까운 정확도 도달 가능

In [11]:
results = model.evaluate(test_data.batch(512), verbose=2)

for name, value in zip(model.metrics_names, results):
  print("%s: %.3f" % (name, value))

49/49 - 2s - loss: 0.3161 - accuracy: 0.8596
loss: 0.316
accuracy: 0.860
